import zipfile
with zipfile.ZipFile('custom.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

import zipfile
with zipfile.ZipFile('./custom/s1.zip', 'r') as zip_ref:
    zip_ref.extractall('./custom/s1')

!pip install imutils pandas tensorflow --index-url https://infyartifactory.ad.infosys.com/artifactory/api/pypi/pypi-remote/simple --trusted-host infyartifactory.ad.infosys.com

In [3]:
# import the necessary packages
from imutils import build_montages
from imutils import paths
import numpy as np
import pandas as pd
import cv2
import os,random
from PIL import Image,ImageOps
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import load_model
import warnings
from tqdm.notebook import tqdm
# for min pooling
from keras import backend as K
warnings.filterwarnings('ignore')

In [4]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

def cv2_imshow(mat):
    plt.imshow(mat)

In [5]:
img1 = cv2.imread('./custom/full/001/001_00.png')
img1.shape

(359, 725, 3)

In [6]:
def make_pairs():
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    pairImages = []
    pairLabels = []
    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    # loop over all images
    df = pd.read_csv('./custom/train_data.csv').sample(frac=0.5)
    path = './custom/full'
    for i in tqdm(range(len(df)//3)):
        img1 = os.path.join(path, df.iloc[i, 0])
        img1 = Image.open(img1)
        img1 = img1.resize((725, 359))
        img1 = ImageOps.grayscale(img1)
        img1 = np.asarray(img1)/255.0
        img2 = os.path.join(path, df.iloc[i, 1])
        img2 = Image.open(img2)
        img2 = img2.resize((725, 359))
        img2 = ImageOps.grayscale(img2)
        img2 = np.asarray(img2)/255.0
        pairImages.append([img1, img2])
        pairLabels.append([df.iloc[i, 2]])
        
    return (np.array(pairImages), np.array(pairLabels))

In [7]:
def make_pairs_test():
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    pairImages = []
    pairLabels = []
    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    # loop over all images
    df = pd.read_csv('./custom/test_data.csv').sample(frac=0.5)
    path = './custom/full'
    for i in tqdm(range(len(df))):
        img1 = os.path.join(path, df.iloc[i, 0])
        img1 = Image.open(img1)
        img1 = img1.resize((725, 359))
        img1 = ImageOps.grayscale(img1)
        img1 = np.asarray(img1)/255.0
        img2 = os.path.join(path, df.iloc[i, 1])
        img2 = Image.open(img2)
        img2 = img2.resize((725, 359))
        img2 = ImageOps.grayscale(img2)
        img2 = np.asarray(img2)/255.0
        pairImages.append([img1, img2])
        pairLabels.append([df.iloc[i, 2]])
        
    return (np.array(pairImages), np.array(pairLabels))

In [8]:
# build the positive and negative image pairs
print("[INFO] preparing positive and negative pairs...")
(pairTest, labelTest) = make_pairs_test()
#(pairTest, labelTest)[0]
#images = []
print('Prepared')

[INFO] preparing positive and negative pairs...


  0%|          | 0/1160 [00:00<?, ?it/s]

Prepared


In [9]:
# build the positive and negative image pairs
print("[INFO] preparing positive and negative pairs...")
(pairTrain, labelTrain) = make_pairs()
#(pairTrain, labelTrain)[0]
#images = []
print('Prepared')

[INFO] preparing positive and negative pairs...


  0%|          | 0/3480 [00:00<?, ?it/s]

Prepared


In [10]:
# specify the shape of the inputs for our network
IMG_SHAPE = (359,725,1)
# specify the batch size and number of epochs
BATCH_SIZE = 32
EPOCHS = 500

In [11]:
# define the path to the base output directory
BASE_OUTPUT = "./output"
# use the base output path to derive the path to the serialized
# model along with training history plot
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "siamese_model"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])

In [12]:
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,
        keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [13]:
def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)

    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)

    # return the computed contrastive loss to the calling function
    return loss

In [14]:
def plot_training(H, plotPath):
    # construct a plot that plots and saves the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.history["loss"], label="train_loss")
    plt.plot(H.history["val_loss"], label="val_loss")
    plt.plot(H.history["accuracy"], label="train_acc")
    plt.plot(H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")

    plt.savefig(plotPath)

In [15]:
# def build_siamese_model(inputShape, embeddingDim=48):
#     # specify the inputs for the feature extractor network
#     inputs = Input(inputShape)
#     # define the first set of CONV => RELU => POOL => DROPOUT layers
#     x = Conv2D(64, (2, 2), padding="same", activation="relu")(inputs)
#     x = MaxPooling2D(pool_size=(2, 2))(x)
#     x = Dropout(0.3)(x)
#     # second set of CONV => RELU => POOL => DROPOUT layers
#     x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
#     x = MaxPooling2D(pool_size=2)(x)
#     x = Dropout(0.3)(x)
#     # prepare the final outputs
#     pooledOutput = GlobalAveragePooling2D()(x)
#     outputs = Dense(embeddingDim)(pooledOutput)
#     # build the model
#     model = Model(inputs, outputs)
#     # return the model to the calling function
#     return model

In [16]:
def build_siamese_model(inputShape, embeddingDim=60):
	# specify the inputs for the feature extractor network
	inputs = Input(inputShape)
	# define the first set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(32, (2, 2), padding="same", activation="relu")(inputs)
	x = -K.pool2d(-x, pool_size=(2, 2), strides=(2, 2))
	# second set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(32, (2, 2), padding="same", activation="relu")(x)
	x = -K.pool2d(-x, pool_size=(2, 2), strides=(2, 2))
    # Third set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
	x =-K.pool2d(-x, pool_size=(2, 2), strides=(2, 2))
    # fourth set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(64, (2, 2), padding="same", activation="relu")(x)
	x = -K.pool2d(-x, pool_size=(2, 2), strides=(2, 2))
    # fifth set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(128, (2, 2), padding="same", activation="relu")(x)
	x = -K.pool2d(-x, pool_size=(2, 2), strides=(2, 2))
    # sixth set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(128, (2, 2), padding="same", activation="relu")(x)
	x = -K.pool2d(-x, pool_size=(2, 2), strides=(2, 2))
    # seventh set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(512, (2, 2), padding="same", activation="relu")(x)
	x = -K.pool2d(-x, pool_size=(2, 2), strides=(2, 2))
    # Eight set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(512, (2, 2), padding="same", activation="relu")(x)
	x = -K.pool2d(-x, pool_size=(2, 2), strides=(2, 2))
  # prepare the final outputs
	pooledOutput = GlobalAveragePooling2D()(x)
	outputs = Dense(embeddingDim)(pooledOutput)
	# build the model
	model = Model(inputs, outputs)
	# return the model to the calling function
	return model

In [ ]:
# configure the siamese network
print("[INFO] building siamese network...")
imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)
featureExtractor = build_siamese_model(IMG_SHAPE)
featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)
# finally, construct the siamese network
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)

# compile the model
print("[INFO] compiling model...")
model.compile(loss='binary_crossentropy', optimizer="adam",#contrastive_loss
    metrics=["accuracy"])
# train the model
print("[INFO] training model...")
history = model.fit(
    [pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
    validation_data=([pairTest[:, 0], pairTest[:, 1]], labelTest[:]),
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS)

# serialize the model to disk
print("[INFO] saving siamese model...")
model.save(MODEL_PATH)
# plot the training history
print("[INFO] plotting training history...")
plot_training(history,PLOT_PATH)

[INFO] building siamese network...
[INFO] compiling model...
[INFO] training model...
Epoch 1/500
109/109 [==============================] - 39s 341ms/step - loss: 0.6921 - accuracy: 0.5466 - val_loss: 0.6909 - val_accuracy: 0.5526
Epoch 2/500
109/109 [==============================] - 34s 316ms/step - loss: 0.6907 - accuracy: 0.5466 - val_loss: 0.6897 - val_accuracy: 0.5526
Epoch 3/500
109/109 [==============================] - 34s 309ms/step - loss: 0.6898 - accuracy: 0.5466 - val_loss: 0.6888 - val_accuracy: 0.5526
Epoch 4/500
109/109 [==============================] - 34s 311ms/step - loss: 0.6894 - accuracy: 0.5466 - val_loss: 0.6884 - val_accuracy: 0.5526
Epoch 5/500
109/109 [==============================] - 34s 313ms/step - loss: 0.6891 - accuracy: 0.5466 - val_loss: 0.6882 - val_accuracy: 0.5526
Epoch 6/500
109/109 [==============================] - 34s 311ms/step - loss: 0.6890 - accuracy: 0.5466 - val_loss: 0.6880 - val_accuracy: 0.5526
Epoch 7/500
109/109 [=================

In [ ]:
path = './custom/full'

fold = random.choice(os.listdir(path)).replace('_forg','')

org_fold = os.listdir(os.path.join(path,str(fold)))

forg_fold = os.listdir(os.path.join(path,str(fold) + '_forg'))

org1 = os.path.join(path,str(fold), random.choice(org_fold))
org2 = os.path.join(path,str(fold), random.choice(org_fold))

forg = os.path.join(path,str(fold) + '_forg', random.choice(forg_fold))


imageA = cv2.imread(org1, 0)
imageB = cv2.imread(org2, 0)

origA = imageA.copy()
origB = imageB.copy()

# add a batch dimension to both images
imageA = np.expand_dims(imageA, axis=0)
imageB = np.expand_dims(imageB, axis=0)

# scale the pixel values to the range of [0, 1]
imageA = imageA / 255.0
imageB = imageB / 255.0

# use our siamese model to make predictions on the image pair,
# indicating whether or not the images belong to the same class
preds = model.predict([imageA, imageB])
proba = preds[0][0]
# initialize the figure
fig = plt.figure("Pair #{}".format(1), figsize=(4, 2))
plt.suptitle("Similarity: {:.2f}".format(proba))

# show first image
ax = fig.add_subplot(1, 2, 1)
plt.imshow(origA, cmap=plt.cm.gray)
plt.axis("off")

# show the second image
ax = fig.add_subplot(1, 2, 2)
plt.imshow(origB, cmap=plt.cm.gray)
plt.axis("off")

print(org1,org1)
# show the plot
plt.show()

In [ ]:
df = pd.read_csv('./custom/test_data.csv').sample(frac=0.5)
df